In [7]:
%conda

IndexError: list index out of range

In [2]:
import osgeo

ModuleNotFoundError: No module named 'osgeo'

In [1]:
# import libraries
import os
import subprocess
import shutil
import fnmatch
from osgeo import gdal
import cv2

ModuleNotFoundError: No module named 'osgeo'

In [ ]:
# function that splits Geotiffs into required pieces

def create_tiles(src_path, dst_path):

    # specify tile dimension
    tile_size_x = 512
    tile_size_y = 512

    # dynamically get the image dimension
    dataset = gdal.Open(src_path)
    band = dataset.GetRasterBand(1)
    xsize = band.XSize
    ysize = band.YSize

    # clip image using using tile_size and gdal_translate iteratively
    for i in range(0, xsize, tile_size_x):
        for j in range(0, ysize, tile_size_y):
            com_string = "gdal_translate -of GTIFF -srcwin " + str(i) + ", " + str(j) + ", " + str(tile_size_x) + ", " + str(tile_size_y) + " " + str(src_path) + " " + str(dst_path) + "_" + str(i) + "_" + str(j) + ".tif"
            os.system(com_string)

In [ ]:
source_path = "/content/drive/MyDrive/geotiffs/"
dest_path = "/content/drive/MyDrive/Split_Geotiffs/"

# splits all geotiff in input path folder
def gtiff_split(src_path, dest_path):
    for file in os.listdir(src_path):
      dest_path1 = os.path.join(dest_path, file[:-4])
      os.mkdir(dest_path1)      
      create_tiles(os.path.join(source_path, file), os.path.join(dest_path1, file[:-4]))
    print(f"success! all files have been split")

In [ ]:
gtiff_split(source_path, dest_path)

success! all files have been split


In [ ]:
# function converts geotiff to png files (8bit)
def geotiff_to_png(input_tiff_img, output_png_img, output_pix_type='Byte', output_format='png', compress='png', percentiles=[2, 98]):

    img_src = gdal.Open(input_tiff_img)

    cmd = ['gdal_translate', '-ot', output_pix_type, '-of', output_format, '-co', compress]

    for band_id in range(img_src.RasterCount):
        band_id = band_id + 1
        band = img_src.GetRasterBand(band_id)

        # cal the band minimum and max values of raster
        b_min = band.GetMinimum()
        b_max = band.GetMaximum()

        # if not exist minimum and maximum values
        if b_min is None or b_max is None:
            (b_min, b_max) = band.ComputeRasterMinMax(1)
            if b_min == b_max:
                b_min -= 1

        cmd.append('-scale_{}'.format(band_id))
        cmd.append('{}'.format(b_min))
        cmd.append('{}'.format(b_max))
        cmd.append('{}'.format(0))
        cmd.append('{}'.format(255))

    cmd.append(input_tiff_img)
    cmd.append(output_png_img)
    print("Conversion command:", cmd)
    subprocess.call(cmd)

In [ ]:
# function that activates the converter wrt geotiff in input path
def converter(input_path, output_path):
    # convert each file to output
    for file in os.listdir(input_path):
        geotiff_to_png(os.path.join(input_path, file), (output_path + file[:-4] + "png"))
    print(f"success! all files have been converted to png")

In [ ]:
# move xml extension files from png_path to its xml_path
def separate_png_xml(src_path, dst_path, ext):
    for file in fnmatch.filter(os.listdir(src_path), ext):
        shutil.move(os.path.join(src_path, file), os.path.join(dst_path, file))
    print(f"success!! xml file has been moved to Palm_xml")

In [ ]:
# directories
geotiff_path = " "
png_path = " "
xml_path = " "

In [ ]:
# Convert the geotiffs to png
converter(input_path=geotiff_path, output_path=png_path)

In [ ]:
# Remove the all xml files from the png directory to xml folder
separate_png_xml(src_path=png_path, dst_path=xml_path, ext='*.xml')